In [1]:
# https://medium.com/analytics-vidhya/object-detection-with-opencv-python-using-yolov3-481f02c6aa35
import cv2

In [2]:
import numpy as np

In [3]:
# yolo v3 알고리즘을 로드
net = cv2.dnn.readNet("yolov3.weights","yolov3.cfg")

classes = []
with open("coco.names","r") as f:
    # coco.names: 80 개의 클래스 이름
    classes = [line.strip() for line in f.readlines()]



In [4]:
# net.getUnconnectedOutLayers 및 
# net.getLayerNames 를 사용하여 감지되는 개체를 정의하는 출력 레이어를 정의

layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [5]:
# 무작위 표본 표출, 난수 만듦
colors= np.random.uniform(0,255,size=(len(classes),3))

In [6]:
# 이미지의 높이와 너비를 40 % 및 30 % 배율로 줄입니다. 
# 그리고 모든 값을 원본 이미지의 높이, 너비, 채널 변수에 저장합니다.
img = cv2.imread("area.jpeg")
img = cv2.resize(img,None,fx=0.4,fy=0.3)
height,width,channels = img.shape

In [7]:
# cv2.imshow("Image",img)
# cv2.waitKey(0)
# cv2.destroyAllWind

In [8]:
# blob: 특징 추출
# cv2.dnn.blobFromImage 를 사용 하고 몇 가지 변수를 전달 하여 blob에서 개체를 감지
blob = cv2.dnn.blobFromImage(img,0.00392,(416,416),(0,0,0),True,crop=False)

# img 는 파일 이름, scalefactor 0.00392, blob에서 사용할 이미지 크기는 (416,416), 
# 레이어에서 평균 빼기 (0,0) 없음 , 0), 

# True 플래그를 설정 하면 OpenCV가 BGR을 사용하기 때문에 
# 파란색을 빨간색으로 반전하지만 이미지에 RGB로 채널이 있음을 의미

In [9]:
# 세 가지 다른 Blob이 어떻게 보이는지 살펴볼 수 있는 코드
# for b in blob:
#     for n,img_blob in enumerate(b):
#         cv2.imshow(str(n),img_blob)

In [10]:
# net.setInput(blob): blob을 네트워크에 전달한 다음, 이를 출력하고 계층에 전달
net.setInput(blob)

# outs 에는 위쪽, 왼쪽, 오른쪽, 아래쪽 위치, 클래스 이름과 같은 
# 개체의 위치를 추출하는 데 필요한 모든 정보가 포함
outs = net.forward(outputlayers)

#print(outs[1])

In [11]:
# 아웃을 화면에 정보를 표시하여,
# 물체를 예측할 때 알고리즘이 얼마나 자신감이 있는지를 의미하는 신뢰도를 예측하고자 함


class_ids=[]
confidences=[]
boxes=[]
for out in outs:
    for detection in out:
        # 신뢰도 예측을 위해 out을 반복 하고 각 out에 대한 모든 점수를 얻음
        scores = detection[5:]
        # 가장 높은 점수를 할당
        # CLASS_ID의 점수를 전달하여 가장 높은 점수에 대한 신뢰도 계산함
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        # 신뢰 수준 임계값을 0.5로 할당한다. 
        # 0.5 이상이면 물체가 감지되었음을 의미.
        if confidence > 0.5:
            # center_x, center_y를 구하고, w 를 너비로, h 를 감지 된 물체의 높이로 설정
            # 이전에 원본 이미지에서 저장 한 높이, 너비 변수를 사용
            center_x= int(detection[0]*width)
            center_y= int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
        
            # 물체가 감지되었다는 증거를 위해 물체 중앙에 두께 2의 원을 그림
            #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
            
            # center_x, center_y, w, h 를 사용하여 
            # 감지된 물체 주위에 사각형을 그림.
            # class_id, 신뢰도 같은 정보 추가
            x=int(center_x - w/2)
            y=int(center_y - h/2)
            #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            
            boxes.append([x,y,w,h]) #put all rectangle areas
            confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
            class_ids.append(class_id) #name of the object tha was detected

In [12]:
# 한 물체에 대해 사각형이 2개 이상 생기는 모습을 볼 수 있음.
# NMS (Non-Max Suppression) 기능을 사용하여 이 현상을 없애고자 함.

# 0.6 미만의 값을 가진 모든 상자를 제거하여 가장 좋은 것만 유지하도록 결정
# indexes 변수는 감지 된 고유한 개체를 추적하여 동일한 물체를 여러번 감지하지 않음.
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

In [13]:
# 모든 상자 에 대해 아래 loop 반복을 사용하여 
# 상자가 색인에 나타나면 사각형을 그리고 색상을 지정하고 클래스 이름의 텍스트를 그 위에 놓습니다.

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        cv2.putText(img,label,(x,y+30),font,1,(255,255,255),2)
            
cv2.imshow("Image",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ->자동차, 사람 및 오토바이를 감지